In [ ]:
from zipfile import ZipFile 

fn = "/content/drive/MyDrive/Movie/1m/ml-1m.zip"
with ZipFile(fn, 'r') as zip:
  zip.extractall()
  print('Extract Complete!!')


Extract Complete!!


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable


In [ ]:
#Dataset Code
m = pd.read_csv('ml-1m/movies.dat', sep='::', header=  None, engine = 'python', encoding = 'latin-1')
u = pd.read_csv('ml-1m//users.dat', sep='::', header=  None, engine = 'python', encoding = 'latin-1')
r = pd.read_csv('ml-1m/ratings.dat', sep='::', header=  None, engine = 'python', encoding = 'latin-1') 

In [ ]:
#Trainign Testing DS

tr_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')

tr_set = np.array(tr_set, dtype = 'int')

ts_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')

ts_set = np.array(tr_set, dtype = 'int')

In [ ]:
#Users and Movies

n_users = int(max((max(tr_set[:, 0]), max(ts_set[:, 0]))))
n_movies = int(max((max(tr_set[:, 1]), max(ts_set[:, 1]))))

In [ ]:
n_movies

1682

In [ ]:
def arrange(data):
  new_data = []
  for users in range(1, n_users+1):
    id_movies = data[:, 1][data[:, 0] == users]
    id_ratings = data[:, 2][data[:, 0] == users]
    ratings = np.zeros(n_movies)
    ratings[id_movies - 1] =  id_ratings
    new_data.append(list(ratings))
  return new_data

In [ ]:
tr_set = arrange(tr_set)
ts_set = arrange(ts_set)

In [ ]:
tr_set = torch.FloatTensor(tr_set)
ts_set = torch.FloatTensor(ts_set)

In [ ]:
#Model Architecture

class SAE(nn.Module):
  def __init__(self, ):
    super(SAE, self).__init__()
  
    self.fc1 = nn.Linear(n_movies, 20)
    self.fc2 = nn.Linear(20, 10)
    self.fc3 = nn.Linear(10, 5)
    self.fc4 = nn.Linear(5, 20)
    self.fc5 = nn.Linear(20, n_movies)
    self.activation = nn.Sigmoid()

  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.activation(self.fc3(x))
    x = self.activation(self.fc4(x))
    x = self.fc5(x)
    return x
  
sae = SAE()

criteria = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay= 0.5)


In [ ]:
#Training

epochs = 200

for epoch in range(1, epochs + 1):
  tr_loss = 0
  s = 0

  for user in range(n_users):
    input =  Variable(tr_set[user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criteria(output, target)
      mean_corrector = n_movies/float(torch.sum(target.data > 0 ) + 1e-10)
      loss.backward()
      tr_loss += np.sqrt(loss.data*mean_corrector)
      s +=1.
      optimizer.step()
  print('epoch: '+ str(epoch) + ' loss: ' + str(tr_loss/s))

In [ ]:
#For Test Data
ts_loss = 0
s = 0.
for user in range(n_users):
    input =  Variable(tr_set[user]).unsqueeze(0)
    target = Variable(ts_set[user])
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[(target == 0).unsqueeze(0)] = 0
      loss = criteria(output, target)
      mean_corrector = n_movies/float(torch.sum(target.data > 0 ) + 1e-10)
      loss.backward()
      ts_loss += np.sqrt(loss.data*mean_corrector)
      s +=1.
      optimizer.step()
print('test loss: ' + str(ts_loss/s))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([1682])) that is different to the input size (torch.Size([1, 1682])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


test loss: tensor(0.9425)
